In [1]:
import pandas as pd
import io
import re
import os

from stops_prn_extractor import StopsPRNExtractor

# Instantiate the extractor object
extractor = StopsPRNExtractor()


In [ ]:
# Instantiate the extractor object
extractor = StopsPRNExtractor()

prn_files_directory = "./stops_output_prn_report_files"

prn_files = [
    ("2024", f"{prn_files_directory}/A2_MBTA-CATA-MWRTA-BATA-MVRTA#MBTA-CATA-MWRTA-BATA-MVRTA#MBTA-CATA-MWRTA-BATA-MVRTA_STOPSY2024Results.prn"),
    ("2045", f"{prn_files_directory}/A2_MBTA-CATA-MWRTA-BATA-MVRTA#MBTA50-CATA-MWRTA-BATA-MVRTA#MBTA50-CATA-MWRTA-BATA-MVRTA_STOPSY2045Results.prn"),
    ("2050", f"{prn_files_directory}/A2_MBTA-CATA-MWRTA-BATA-MVRTA#MBTA50-CATA-MWRTA-BATA-MVRTA#MBTA50-CATA-MWRTA-BATA-MVRTA_STOPSY2050Results.prn"),
    ("example", f"{prn_files_directory}/example.prn"),
    # Add more as needed
]


# Add files to the extractor
extractor.add_files(prn_files)

# Extract all tables at once
extractor.extract_all_tables()

# Display results (optional, good for debugging)
for alias_key in extractor.tables:
    print(f"\nAlias: {alias_key}")
    print("Metadata:")
    for k, v in extractor.metadata[alias_key].items():
        print(f"  {k}: {v}")
    print(f"\nTable data for {alias_key}:")
    display(extractor.tables[alias_key])

# Export extracted tables to CSV files
extractor.export_to_csv(output_dir="./extracted_csv_tables")


Alias: 2024_table_9_01
Metadata:
  Run: Boston_Regional_STOPS                                                                                            10:48:37
  Program: STOPS

Table data for 2024_table_9_01:


,Stop_id1,Station_Name,Y2024_EXISTING_WLK,Y2024_EXISTING_KNR,Y2024_EXISTING_PNR,Y2024_EXISTING_XFR,Y2024_EXISTING_ALL,Y2024_NO-BUILD_WLK,Y2024_NO-BUILD_KNR,Y2024_NO-BUILD_PNR,Y2024_NO-BUILD_XFR,Y2024_NO-BUILD_ALL,Y2024_BUILD_WLK,Y2024_BUILD_KNR,Y2024_BUILD_PNR,Y2024_BUILD_XFR,Y2024_BUILD_ALL
0,A6456&T,Alewife,2485,460,202,824,3971,2485,460,202,824,3971,2485,460,202,824,3971
1,A6457&T,Davis,4046,229,0,512,4788,4046,229,0,512,4788,4046,229,0,512,4788
2,A6459&T,Porter,3832,213,0,1441,5485,3832,213,0,1441,5485,3832,213,0,1441,5485
3,A6461&T,Harvard,6203,172,0,1554,7929,6203,172,0,1554,7929,6203,172,0,1554,7929
4,A6463&T,Central,6570,214,0,711,7495,6570,214,0,711,7495,6570,214,0,711,7495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11327,A6639&T,Assembly,2163,30,0,0,2192,2163,30,0,0,2192,2163,30,0,0,2192
11328,A6428&T,Wellington,1921,367,1630,902,4819,1921,367,1630,902,4819,1921,367,1630,902,4819
11329,A6430&T,Malden Center,5680,487,5,2743,8916,5680,487,5,2743,8916,5680,487,5,2743,8916
11330,A6432&T,Oak Grove,2073,520,51,785,3429,2073,520,51,785,3429,2073,520,51,785,3429



Alias: 2024_table_10_01
Metadata:
  Run: Boston_Regional_STOPS                                                                                            10:48:37
  Program: STOPS

Table data for 2024_table_10_01:


,Route_ID,Route_Name,Count,Y2024_EXISTING_WLK,Y2024_EXISTING_KNR,Y2024_EXISTING_PNR,Y2024_EXISTING_ALL,Y2024_NO-BUILD_WLK,Y2024_NO-BUILD_KNR,Y2024_NO-BUILD_PNR,Y2024_NO-BUILD_ALL,Y2024_BUILD_WLK,Y2024_BUILD_KNR,Y2024_BUILD_PNR,Y2024_BUILD_ALL
0,04N&M,--RT04N-RT04N-Waverly St Natic,0,299,6,2,307,299,6,2,307,299,6,2,307
1,04S&M,--RT04S-RT04S-Central Hub Beav,0,147,2,1,150,147,2,1,150,147,2,1,150
2,07C&M,--RT07C-RT07C,0,255,26,4,285,255,26,4,285,255,26,4,285
3,1&M,--RT01-RT01-Woodland Green Lin,0,144,4,2,150,144,4,2,150,144,4,2,150
4,1&T,--1-Harvard Square - Nubian St,9490,9361,40,172,9573,9361,40,172,9573,9361,40,172,9573
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,Mattapan&T,--Mattapan Trolley-Mattapan Tr,4523,3416,289,896,4601,3416,289,896,4601,3416,289,896,4601
242,NCS&M,--Natick Commuter Shuttle-Nati,0,8,0,1,10,8,0,1,10,8,0,1,10
243,Orange&T,--Orange Line-Orange Line-Rapi,115769,93982,8294,12794,115071,93982,8294,12794,115071,93982,8294,12794,115071
244,Red&T,--Red Line-Red Line-Rapid Tran,133995,109776,8110,12642,130528,109776,8110,12642,130528,109776,8110,12642,130528



Alias: 2024_table_10_02
Metadata:
  Run: Boston_Regional_STOPS                                                                                            10:48:37
  Program: STOPS

Table data for 2024_table_10_02:


...


Alias: example_table_11_03
Metadata:
  Run: Boston_Regional_STOPS                                                                                            15:01:23
  Program: STOPS

Table data for example_table_11_03:


,HH_Cars,Sub_mode,Access_mode,Y2024_EXISTING_Model,Y2024_EXISTING_Survey,Y2050_NO_BUILD_Model,Y2050_NO_BUILD_Survey,Y2050_BUILD_Model,Y2050_BUILD_Survey,Y2050_BUILD_Project_Model,Y2050_BUILD_Project_Survey
0,0 car H,Fixed Guideway Only,Walk Access,13651,0,15867,0,15867,0,0,0
1,0 car H,Fixed Guideway Only,KNR Access,1420,0,1809,0,1809,0,0,0
2,0 car H,Fixed Guideway Only,PNR Access,42,0,66,0,66,0,0,0
3,0 car H,Fixed Guideway Only,All Access,15113,0,17742,0,17742,0,0,0
4,0 car H,Fixed Guideway+Bus,Walk Access,1406,0,2458,0,2458,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
77,All car,All Fixed Guideway,KNR Access,5677,0,6731,0,6731,0,0,0
78,All car,All Fixed Guideway,PNR Access,2852,0,4788,0,4788,0,0,0
79,All car,All Fixed Guideway,All Access,46424,0,55754,0,55754,0,0,0
80,. . . . .,. . . . . . . . . .,. . . . . . .,0,0,0,0,0,0,0,0



Alias: example_table_11_04
Metadata:
  Run: Boston_Regional_STOPS                                                                                            15:01:23
  Program: STOPS

Table data for example_table_11_04:


,HH_Cars,Sub_mode,Access_mode,Y2024_EXISTING_Model,Y2024_EXISTING_Survey,Y2050_NO_BUILD_Model,Y2050_NO_BUILD_Survey,Y2050_BUILD_Model,Y2050_BUILD_Survey,Y2050_BUILD_Project_Model,Y2050_BUILD_Project_Survey
0,0 car H,Fixed Guideway Only,Walk Access,112229,0,140112,0,40112,0,0,0
1,0 car H,Fixed Guideway Only,KNR Access,7073,0,9045,0,9045,0,0,0
2,0 car H,Fixed Guideway Only,PNR Access,656,0,1081,0,1081,0,0,0
3,0 car H,Fixed Guideway Only,All Access,119957,0,150238,0,50238,0,0,0
4,0 car H,Fixed Guideway+Bus,Walk Access,22819,0,35677,0,35677,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
76,All car,All Fixed Guideway,Walk Access,299968,0,382773,0,82773,0,0,0
77,All car,All Fixed Guideway,KNR Access,32523,0,42263,0,42263,0,0,0
78,All car,All Fixed Guideway,PNR Access,78460,0,115390,0,15390,0,0,0
79,All car,All Fixed Guideway,All Access,410950,0,540425,0,40425,0,0,0


Exported '2024_table_9_01' to './extracted_csv_tables\Table_9_01\[2024]__Table_9_01.csv' successfully.
Exported '2024_table_10_01' to './extracted_csv_tables\Table_10_01\[2024]__Table_10_01.csv' successfully.
Exported '2024_table_10_02' to './extracted_csv_tables\Table_10_02\[2024]__Table_10_02.csv' successfully.
Exported '2024_table_11_01' to './extracted_csv_tables\Table_11_01\[2024]__Table_11_01.csv' successfully.
Exported '2024_table_11_02' to './extracted_csv_tables\Table_11_02\[2024]__Table_11_02.csv' successfully.
Exported '2024_table_11_03' to './extracted_csv_tables\Table_11_03\[2024]__Table_11_03.csv' successfully.
Exported '2024_table_11_04' to './extracted_csv_tables\Table_11_04\[2024]__Table_11_04.csv' successfully.
Exported '2045_table_9_01' to './extracted_csv_tables\Table_9_01\[2045]__Table_9_01.csv' successfully.
Exported '2045_table_10_01' to './extracted_csv_tables\Table_10_01\[2045]__Table_10_01.csv' successfully.
Exported '2045_table_10_02' to './extracted_csv_tabl